In [3]:
import boto3
from dotenv import load_dotenv
import os
import csv

# Cargar variables de entorno
load_dotenv()

aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_default_region = os.getenv("AWS_DEFAULT_REGION")

boto3.setup_default_session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_default_region
)

dynamodb = boto3.client('dynamodb')

table_name = 'User-7kkcm5dn2rb77hst5nh7gbdisa-staging'

# Obtener una muestra para determinar todas las columnas
response = dynamodb.scan(TableName=table_name, Limit=1)
if not response.get('Items'):
    print("La tabla está vacía")
    exit()

# Obtener todas las columnas de la primera fila
columns_to_extract = list(response['Items'][0].keys())

# Exportar a CSV
with open('dynamodb_users.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    last_evaluated_key = None

    # Escribimos la cabecera del archivo CSV con todas las columnas
    writer.writerow(columns_to_extract)

    while True:
        if last_evaluated_key:
            response = dynamodb.scan(
                TableName=table_name,
                ExclusiveStartKey=last_evaluated_key
            )
        else:
            response = dynamodb.scan(TableName=table_name)

        for item in response['Items']:
            row = []
            for column in columns_to_extract:
                value_obj = item.get(column, {})

                # Si es String
                if "S" in value_obj:
                    value = value_obj["S"]
                # Si es Number
                elif "N" in value_obj:
                    value = value_obj["N"]
                # Si es Binary
                elif "B" in value_obj:
                    value = value_obj["B"]
                # Si es una Lista
                elif "L" in value_obj:
                    value = [v.get("S") or v.get("N") or v.get("B") or "" for v in value_obj["L"]]
                    value = ", ".join(value)  # Convertimos la lista en una cadena legible
                # Si es un Mapa
                elif "M" in value_obj:
                    value = str(value_obj["M"])  # Convertimos el mapa a string
                else:
                    value = ""

                row.append(value)

            writer.writerow(row)

        if 'LastEvaluatedKey' in response:
            last_evaluated_key = response['LastEvaluatedKey']
        else:
            break

print("Datos exportados a dynamodb_users.csv")


Datos exportados a dynamodb_users.csv
